In [1]:
import gensim
import numpy as np

In [3]:
wv_model_path = "/home/v2john/Documents/GoogleNews-vectors-negative300.bin.gz"
wv_model = gensim.models.KeyedVectors.load_word2vec_format(wv_model_path, binary=True)

In [4]:
affect_intensity_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/" + \
    "lexicons/NRC-AffectIntensity-Lexicon.txt"

In [5]:
from nltk.corpus import stopwords

def remove_stopwords(string):
    split_string = \
        [word for word in string.split()
         if word not in stopwords.words('english')]
    
    return " ".join(split_string)

In [6]:
import re

def clean_str(string):
    string = re.sub(r"@[A-Za-z0-9_s(),!?\'\`]+", "", string) # removing any twitter handle mentions
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)

    return remove_stopwords(string.strip().lower())

In [7]:
class Tweet(object):

    def __init__(self, id, text, emotion, intensity):
        self.id = id
        self.text = text
        self.emotion = emotion
        self.intensity = intensity

    def __repr__(self):
        return \
            "id: " + self.id + \
            ", text: " + self.text + \
            ", emotion: " + self.emotion + \
            ", intensity: " + self.intensity

In [8]:
def read_training_data(training_data_file_path):

    train_list = list()
    with open(training_data_file_path) as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            train_list.append(Tweet(array[0], clean_str(array[1]), array[2], float(array[3])))
    return train_list
            
def read_test_data(training_data_file_path):

    test_list = list()
    with open(training_data_file_path) as input_file:
        for line in input_file:
            line = line.strip()
            array = line.split('\t')
            test_list.append(Tweet(array[0], clean_str(array[1]), array[2], None))
    return test_list

In [9]:
emotion = "fear"

In [10]:
training_data_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/dataset/" + \
    emotion + "-ratings-0to1.train.txt"
test_data_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/dataset/" + \
    emotion + "-ratings-0to1.dev.target.txt"
predictions_file_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/predictions/" + \
    emotion + "-pred.txt"
gold_set_path = \
    "/home/v2john/MEGA/Academic/Masters/UWaterloo/Research/WASSA-Task/dataset/gold-set/" + \
    emotion + "-ratings-0to1.dev.gold.txt"

In [11]:
word_list = list()
word_intensities = dict()

with open(affect_intensity_file_path) as affect_intensity_file:
    for line in affect_intensity_file:
        word_int_array = line.replace("\n", "").split("\t")
        
        if (word_int_array[2] == emotion):
            word_list.append(word_int_array[0])
            word_intensities[word_int_array[0]] = float(word_int_array[1])
            
print(len(word_list))

1765


In [12]:
training_tweets = read_training_data(training_data_file_path)

score_train = list()
tweet_train = list()
for tweet in training_tweets:
    tweet_train.append(tweet.text)
    score_train.append(float(tweet.intensity))
    
print(len(score_train))

1147


In [13]:
x_train = list()

for tweet in tweet_train:
    x_vector = list()
    score = 0.0
    for word in word_list:
        if word in tweet:
            score += tweet.count(word) * float(word_intensities[word])
    x_vector.append(score)
    x_vector.append(score**2)
    x_vector.append(score**3)
    x_vector.append(score**4)
#     print(len(x_vector))
    
    vector_list = list()
    for word in tweet.split():
        try:
            vector_list.append(wv_model[word])
        except Exception as e:
            pass
    
    if len(vector_list) == 0:
        vec_rep = np.zeros(300).tolist()
    else:
        vec_rep = sum(vector_list) / float(len(vector_list))
#     print(len(vec_rep))
    
    x_vector.extend(vec_rep)
    x_train.append(x_vector)

print(len(x_train[0]))

304


In [14]:
from sklearn import model_selection, ensemble, svm

scores = \
    model_selection.cross_val_score(
        ensemble.GradientBoostingRegressor(), x_train, 
        score_train, cv=10, scoring='neg_mean_squared_error'
    )

mean_score = scores.mean()
print("Accuracy: %0.2f (+/- %0.2f)" % (mean_score, scores.std() * 2))

Accuracy: -0.03 (+/- 0.05)


In [15]:
test_tweets = read_test_data(test_data_file_path)
tweet_test = list()
for tweet in test_tweets:
    tweet_test.append(tweet.text)

print(len(tweet_test))

x_test = list()

for tweet in tweet_test:
    x_vector = list()
    score = 0.0
    for word in word_list:
        if word in tweet:
            score += tweet.count(word) * float(word_intensities[word])
    x_vector.append(score)
    x_vector.append(score**2)
    x_vector.append(score**3)
    x_vector.append(score**4)

    vector_list = list()
    for word in tweet.split():
        try:
            vector_list.append(wv_model[word])
        except Exception as e:
            pass
    
    if len(vector_list) == 0:
        vec_rep = np.zeros(300).tolist()
    else:
        vec_rep = sum(vector_list) / float(len(vector_list))
        
    x_vector.extend(vec_rep)
    x_test.append(x_vector)

print(len(x_test))

110
110


In [16]:
ml_model = ensemble.GradientBoostingRegressor()
ml_model.fit(x_train, score_train)

y_test = ml_model.predict(X=x_test)

In [17]:
with open(predictions_file_path, 'w') as predictions_file:
    for i in range(len(y_test)):
        predictions_file.write(
            str(test_tweets[i].id) + "\t" + test_tweets[i].text + "\t" +
            test_tweets[i].emotion +"\t" + str(y_test[i]) + "\n"
        )

In [297]:
for i in range(10):
    print(tweet_test[i], y_test[i])

know going one nights takes act god fall asleep 0.398814208774
horrible lewis dunk begun networking neo geo holiday home mexico 0.42345414724
speaking ex cobblers , saw ricky holmes charlton last week tracking back amp defending dread seeing gorre ball 0.440753717854
ball watching amp rojo 'd header equally dreadful ! ! 0.406487457879
really jumanji 2 w rock , jack black , kevin hart kidding ! wtf ! thisisaterribleidea 0.433469326101
really jumanji 2 w rock , jack black , kevin hart kidding ! wtf ! thisisaterribleidea horrible 0.451750433529
losing villa ' things people worry things wo n't even matter months ' 0.587339054321
worrying worried \? n1peter 5 7 nthrow worry , cares faith leadership worry mindfulness success 0.544449071442
concerns amp anxiety n't matter shall return favor eyematter 0.493953281018
goes butterflies stomach nervous anxietyproblems 0.696920927123


# Simple Neural Network Implementation in PyTorch

In [120]:
import torch
import torch.autograd
import torch.nn.functional as F
from torch.autograd import Variable

print(torch.cuda.is_available())
torch.cuda.set_device(0)

True


In [121]:
dtype = torch.FloatTensor

x_train_tensor = torch.from_numpy(np.array(x_train)).type(dtype)
y_train_tensor = torch.from_numpy(np.array(score_train)).type(dtype)
x_test_tensor = torch.from_numpy(np.array(x_test)).type(dtype)
print(type(x_train_tensor), type(y_train_tensor), type(x_test_tensor))

<class 'torch.FloatTensor'> <class 'torch.FloatTensor'> <class 'torch.FloatTensor'>


In [135]:
_, dim_size = (np.array(x_train).shape)
input_size = dim_size
output_size = 1
num_epochs = 100000
learning_rate = 0.001

In [136]:
import torch.nn as nn
from torch.nn.parameter import Parameter

class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.weight = Parameter(torch.FloatTensor(output_size, input_size))
    
    def forward(self, x):
        out = self.linear(x)
        return out

model = LinearRegression(input_size, output_size)
# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [140]:
for epoch in range(num_epochs):
    # Convert numpy array to torch Variable
    inputs = Variable(x_train_tensor)
    targets = Variable(y_train_tensor)

    # Forward + Backward + Optimize
    optimizer.zero_grad()  
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10000 == 0:
        print ('Epoch [%d/%d], Loss: %.4f' 
               %(epoch+1, num_epochs, loss.data[0]))
        
# Plot the graph
predicted = model(Variable(x_test_tensor)).data.numpy()

Epoch [10000/100000], Loss: 0.0110
Epoch [20000/100000], Loss: 0.0107
Epoch [30000/100000], Loss: 0.0105
Epoch [40000/100000], Loss: 0.0104
Epoch [50000/100000], Loss: 0.0102
Epoch [60000/100000], Loss: 0.0100
Epoch [70000/100000], Loss: 0.0099
Epoch [80000/100000], Loss: 0.0098
Epoch [90000/100000], Loss: 0.0096
Epoch [100000/100000], Loss: 0.0095


In [141]:
with open(predictions_file_path, 'w') as predictions_file:
    for i in range(len(y_test)):
        predictions_file.write(
            str(test_tweets[i].id) + "\t" + test_tweets[i].text + "\t" +
            test_tweets[i].emotion +"\t" + str(y_test[i]) + "\n"
        )

In [18]:
from sklearn import metrics

y_gold = list()
with open(gold_set_path) as gold_set_file:    
    for line in gold_set_file:
        line = line.replace("\n", "")
        split_line = line.split("\t")
        y_gold.append(float(split_line[3]))
# 0.0185 -> previous
print(metrics.mean_squared_error(y_pred=y_test, y_true=y_gold))

0.0234920574774


# Evaluation based on Pearson and Spearman co-efficients

In [ ]:
import numpy
import sys
import scipy.stats

def evaluate(pred,gold):
    
    f=open(pred, "rb")
    pred_lines=f.readlines()
    f.close()
    
    f=open(gold, "rb")
    gold_lines=f.readlines()
    f.close()
    

    if(len(pred_lines)==len(gold_lines)):       
        # align tweets ids with gold scores and predictions
        data_dic={}
        
        for line in gold_lines:
            line = line.decode()
            parts=line.split('\t')
            if len(parts)==4:   
                data_dic[int(parts[0])]=[float(line.split('\t')[3])]
            else:
                raise ValueError('Format problem.')
        
        
        for line in pred_lines:
            line = line.decode()
            parts=line.split('\t')
            if len(parts)==4:  
                if int(parts[0]) in data_dic:
                    try:
                        data_dic[int(parts[0])].append(float(line.split('\t')[3]))
                    except ValueError:
                        # Invalid predictions are replaced by a default value
                        data_dic[int(parts[0])].append(0.5)
                else:
                    raise ValueError('Invalid tweet id.')
            else:
                raise ValueError('Format problem.')
            
            
        
        # lists storing gold and prediction scores
        gold_scores=[]  
        pred_scores=[]
         
        
        # lists storing gold and prediction scores where gold score >= 0.5
        gold_scores_range_05_1=[]
        pred_scores_range_05_1=[]
         
            
        for id in data_dic:
            if(len(data_dic[id])==2):
                gold_scores.append(data_dic[id][0])
                pred_scores.append(data_dic[id][1])
                if(data_dic[id][0]>=0.5):
                    gold_scores_range_05_1.append(data_dic[id][0])
                    pred_scores_range_05_1.append(data_dic[id][1])
            else:
                raise ValueError('Repeated id in test data.')
                
      
        # return zero correlation if predictions are constant
        if numpy.std(pred_scores)==0 or numpy.std(gold_scores)==0:
            return (0,0,0,0)
        

        pears_corr=scipy.stats.pearsonr(pred_scores,gold_scores)[0]                                    
        spear_corr=scipy.stats.spearmanr(pred_scores,gold_scores)[0]   


        pears_corr_range_05_1=scipy.stats.pearsonr(pred_scores_range_05_1,gold_scores_range_05_1)[0]                                    
        spear_corr_range_05_1=scipy.stats.spearmanr(pred_scores_range_05_1,gold_scores_range_05_1)[0]           
        
      
        return (pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1)
                                           
                          
        
    else:
        raise ValueError('Predictions and gold data have different number of lines.')

In [29]:
print("pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1")
print(evaluate(predictions_file_path, gold_set_path))

pears_corr,spear_corr,pears_corr_range_05_1,spear_corr_range_05_1
(0.56360946502302101, 0.50732805258768177, 0.53513370142441596, 0.52117313550595246)
